<a href="https://colab.research.google.com/github/trybol/matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install hyperopt

In [17]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train  = train['features'], train['labels']
X_test, y_test  = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax( y_test, axis=1 )

  y_pred_prob = model_trained.predict( X_test )
  y_pred = np.argmax( y_pred_prob, axis=1 )

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
 return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    Dropout(0.3),


    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
 ])

#train_and_predict( get_cnn_v5(input_shape, num_classes))

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.8078 - accuracy: 0.2437 - val_loss: 1.5157 - val_accuracy: 0.5052
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.2480 - accuracy: 0.5903 - val_loss: 0.5030 - val_accuracy: 0.8255
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.6093 - accuracy: 0.8036 - val_loss: 0.1817 - val_accuracy: 0.9421
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4013 - accuracy: 0.8724 - val_loss: 0.1125 - val_accuracy: 0.9656
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2895 - accuracy: 0.9097 - val_loss: 0.0662 - val_accuracy: 0.9805


In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.1784 - accuracy: 0.9501


0.9501133561134338

In [0]:
def get_model(params):
 return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    Dropout(params['dropouts_cnn_block_one']),


    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    Dropout(params['dropouts_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropouts_cnn_block_three']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropouts_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropouts_dense_block_two']),

    Dense(num_classes, activation='softmax'),
 ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))


  return {'loss' : -accuracy, 'status': STATUS_OK, 'model': model}


In [31]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropouts_cnn_block_one': hp.uniform('dropouts_cnn_block_one', 0.3, 0.6),
    'dropouts_cnn_block_two': hp.uniform('dropouts_cnn_block_two', 0.3, 0.6),
    'dropouts_cnn_block_three': hp.uniform('dropouts_cnn_block_three', 0.3, 0.6),

    'dropouts_dense_block_one': hp.uniform('dropouts_dense_block_one', 0.3, 0.7),
    'dropouts_dense_block_two': hp.uniform('dropouts_dense_block_two', 0.3, 0.7)

}

best = fmin(
     func_obj,
     space,
     tpe.suggest,
     30,
     Trials()
)

{'batch_size': 100.0, 'dropouts_cnn_block_one': 0.5797578020061288, 'dropouts_cnn_block_three': 0.5599664021892918, 'dropouts_cnn_block_two': 0.5073765473342446, 'dropouts_dense_block_one': 0.49208962713977306, 'dropouts_dense_block_two': 0.567354409332488}
accuracy=0.6517006754875183
{'batch_size': 160.0, 'dropouts_cnn_block_one': 0.5706506769267904, 'dropouts_cnn_block_three': 0.4552808425195367, 'dropouts_cnn_block_two': 0.3169445321707244, 'dropouts_dense_block_one': 0.5943218444434141, 'dropouts_dense_block_two': 0.6108072045978639}
accuracy=0.7469387650489807
{'batch_size': 180.0, 'dropouts_cnn_block_one': 0.38462809064376424, 'dropouts_cnn_block_three': 0.5620782323368023, 'dropouts_cnn_block_two': 0.5348869688946214, 'dropouts_dense_block_one': 0.5160579732288135, 'dropouts_dense_block_two': 0.5635792571646684}
accuracy=0.6836734414100647
{'batch_size': 160.0, 'dropouts_cnn_block_one': 0.4547339052988564, 'dropouts_cnn_block_three': 0.4820410647651434, 'dropouts_cnn_block_two':